# Liste doublement chainée

## Principe 

La liste doublement chainée étend le concept de liste en ajoutant un lien vers l'élément précédent à chaque maillon de la chaine.

In [1]:
class Maillon:
    def __init__(self, v, s = None, p = None):
        self.donnee = v
        self.suivant = s 
        self.precedent = p 

        if s: s.precedent = self 
        if p: p.suivant = self
            
    def __str__(self): 
        return "{}".format(self.donnee)

Les arguments optionnels `s` et `p` permettent de spécifier à la construction les éléments suivants et précédents. 

In [2]:
# caché dans les slides

def __liste_double_str__(L):
    if L.empty(): return "⌀"
    m = L._Liste__tq.suivant
    s = "⌀ ← "
    while m != L._Liste__tq:
        s += m.__str__()
        s += " ⇄ " if m.suivant != L._Liste__tq else " → ⌀"
        m = m.suivant
    return s

Comme pour la liste simple, nous écrivons une classe `Liste` qui a pour attributs la tête et la queue de la liste, plus éventuellement la taille. On pourrait l'écrire

In [3]:
class Liste:
    def __init__(self):
        self.tete = None
        self.queue = None
        self.N = 0

Mais comme pour la liste simple, on a avantage à utiliser des maillons vides aux extrémités. Il en suffit d'un que l'on appelle ici `__tq`. Dès lors, 

* l'élément de tête est `__tq.suivant`
* l'élément de queue est `__tq.precedent`

Dans une liste vide, `__tq.suivant = __tq.precedent = __tq`. 

In [4]:
class MaillonVide:
    def __init__(self):
        self.suivant = self
        self.precedent = self
        
class Liste:
    def __init__(self):
        self.__tq = MaillonVide()
        self.__N = 0
        
    __str__ = __liste_double_str__

## Fonctions d'itération

Avant toute chose, écrivons les fonctions d'itération qui nous simplifieront grandement la tâche. 

In [5]:
def debut(L):     return L._Liste__tq.suivant

def fin(L):       return L._Liste__tq

def suivant(m):   return m.suivant

def precedent(m): return m.precedent

def get_val(m):   return m.donnee

def set_val(m,v): m.donnee = v

Par rapport à une liste simplement chainée, notons les différences

* `fin(L)` retourne le maillon vide et pas un lien nul. 


* `precedent(L)` permet d'itérer dans les deux sens. Il est équivalent à `std::list<T>::iterator::operator--()` en C++. 

## Opérations

Les opérations essentielles sont 

* indiquer la taille, indiquer si la liste est vide

* insérer et supprimer en position quelconque

* insérer et supprimer en tête

* insérer et supprimer en queue

* traverser et itérer en avant ou en arrière

## Taille et vide 

Une liste est vide si `début` et `fin` sont identiques (`fin` n'est pas inclut dans la liste)

In [6]:
def est_vide(L):
    return debut(L) == fin(L)
Liste.empty = est_vide

Retourner la taille d'une liste requiert en général de parcourir toute la liste pour compter les maillons, une opération $\Theta(n)$

Ici on a choisi de stocker et maintenir cet information comme attribut de `Liste`

In [7]:
def taille(L):
    return L._Liste__N 
Liste.size = taille

## Insertion avant un maillon `M`

* créer un nouveau maillon `N` 
* relier `N` à gauche au maillon précédent `M`
    * `N.precedent = M.precedent`
    * `M.precedent.suivant = N`
* relier `N` à droite à `M`
    * `N.suivant = M`
    * `M.precedent = N`

In [8]:
def inserer_avant(M,val):
    N = Maillon(val,p = M.precedent, s = M)
    
def inserer_avant_en_liste(L,M,val):
    inserer_avant(M,val)
    L._Liste__N += 1
    
Liste.insert = inserer_avant_en_liste

## Suppression d'un maillon M

* relier `M.precedent` et `M.suivant`
    * `M.precedent.suivant = M.suivant`
    * `M.suivant.precedent = M.precedent`
* selon le langage, détruire `M`


In [9]:
def supprimer(M):
    assert(M and M.precedent and M.suivant)
    M.precedent.suivant = M.suivant
    M.suivant.precedent = M.precedent
    
def supprimer_en_liste(L,M):
    supprimer(M)
    L._Liste__N -= 1

Liste.erase = supprimer_en_liste

## Opérations en tête

Elles peuvent être écrites en une ligne en utilisant les insertions et suppressions quelconques et la fonction d'itération `debut`

In [10]:
def inserer_en_tete(L,val):
    L.insert(debut(L),val)  
    
Liste.appendleft = inserer_en_tete

In [11]:
def supprimer_en_tete(L):
    if L.empty(): raise IndexError()
    L.erase(debut(L))
    
Liste.popleft = supprimer_en_tete

Testons leurs effets

In [12]:
L = Liste(); print(L)
for i in range(3):
    L.appendleft(i**2)
    print(L)

⌀
⌀ ← 0 → ⌀
⌀ ← 1 ⇄ 0 → ⌀
⌀ ← 4 ⇄ 1 ⇄ 0 → ⌀


In [13]:
print(L)
for i in range(4):
    L.popleft()
    print(L)

⌀ ← 4 ⇄ 1 ⇄ 0 → ⌀
⌀ ← 1 ⇄ 0 → ⌀
⌀ ← 0 → ⌀
⌀


IndexError: 

## Opérations en queue

Elles peuvent être écrites en une ligne en utilisant les insertions et suppressions quelconques. On insère devant `fin(L)` et on supprime le maillon `precedent(fin(L))`

In [14]:
def inserer_en_queue(L,val):
    L.insert(fin(L),val)
    
Liste.append = inserer_en_queue

In [15]:
def supprimer_en_queue(L):
    if L.empty(): raise IndexError()
    L.erase(precedent(fin(L)))
Liste.pop = supprimer_en_queue

Testons leurs effets

In [16]:
L = Liste(); print(L)
for i in range(3):
    L.append(i*2)
    print(L)

⌀
⌀ ← 0 → ⌀
⌀ ← 0 ⇄ 2 → ⌀
⌀ ← 0 ⇄ 2 ⇄ 4 → ⌀


In [17]:
for i in range(4):
    L.pop()
    print(L)

⌀ ← 0 ⇄ 2 → ⌀
⌀ ← 0 → ⌀
⌀


IndexError: 

## Itérer

Nous avons fourni plus tôt les fonctions d'itération standards. La principale nouveauté est l'apparition d'une fonction `precedent` qui permet de se déplacer de droite à gauche. Voyons quelques exemples d'utilisation

In [18]:
def afficher_liste(L):
    m = debut(L)
    while m != fin(L):
        print(m, end="")
        if suivant(m) != fin(L): print(end = " ⇄ ")
        m = suivant(m)

In [19]:
T = [ 1, 3, 5, 7 ]; L = Liste()
for t in T: L.append(t)

afficher_liste(L)

1 ⇄ 3 ⇄ 5 ⇄ 7

In [20]:
def supprimer_decroissances(L):
    m = debut(L); prec = get_val(m)
    m = suivant(m)
    
    while m != fin(L):
        if get_val(m) < prec:
            tmp = suivant(m)
            L.erase(m); m = tmp
        else:
            prec = get_val(m)
            m = suivant(m)

In [21]:
T = [ 1, 2, 3, 3, 2, 1, 2, 3, 4, 5, 6, 5, 4, 5, 6, 7, 8 ]; L = Liste()
for t in T: L.append(t)

supprimer_decroissances(L); print(L)

⌀ ← 1 ⇄ 2 ⇄ 3 ⇄ 3 ⇄ 3 ⇄ 4 ⇄ 5 ⇄ 6 ⇄ 6 ⇄ 7 ⇄ 8 → ⌀


In [22]:
def tri_par_insertion(L):
    if L.size() < 2: return
    
    k = suivant(debut(L))
    while k != fin(L):
        tmp = get_val(k)
        i = k
        while i != debut(L) and tmp < get_val(precedent(i)):
            set_val(i,get_val(precedent(i)))
            i = precedent(i)
        set_val(i,tmp)
        k = suivant(k)

In [23]:
T = [ 4, 2, 7, 5, 6, 9, 1, 3, 8 ]; L = Liste()
for t in T: L.append(t)

tri_par_insertion(L); print(L)

⌀ ← 1 ⇄ 2 ⇄ 3 ⇄ 4 ⇄ 5 ⇄ 6 ⇄ 7 ⇄ 8 ⇄ 9 → ⌀


Il est aussi possible de créer des itérateurs inversés

In [24]:
def r_debut(L):     return L._Liste__tq.precedent

def r_fin(L):       return L._Liste__tq

def r_suivant(m):   return m.precedent

def r_precedent(m): return m.suivant

Voyons un exemple d'utilisation

In [25]:
def afficher_inverse(L):
    m = r_debut(L)
    while m != r_fin(L):
        print(m, end=" ")
        m = r_suivant(m)

In [26]:
T = [ 4, 2, 7, 5, 6, 9, 1, 3, 8 ]; L = Liste()
for t in T: L.append(t)
    
afficher_inverse(L)

8 3 1 9 6 5 7 2 4 

## Conclusions

Une liste doublement chainée utilise comme maillons des structures stockant individuellement

* un élément
* un lien vers l'élément suivant
* un lien vers l'élément précédent

Les opérations efficaces en Θ(1) sont

* insertion et suppression en tête et en queue
* insertion et suppression en une position connue

Il n'y a ni pas d'accès indexé. 

L'utilisation d'un maillon vide qui boucle la liste en tête et en queue - ainsi que celle d'itérateurs - simplifie la mise en oeuvre et l'utilisation de la structure.

<table style="width: 100%; border: 0px">
<tr style="background-color:white; border:0px">
<td style="width: 120px; border: 0px">
    <img src="https://heig-vd.ch/ResourcePackages/WhiteFox/assets/images/logo-heig-vd.svg" height=200px align=left >
    </td>
    <td style="vertical-align: middle; border: 0px" height=200px>
    <p style="text-align: left">
        <a href="https://ocuisenaire.github.io/ASD1-notebooks/">ASD1 Notebooks on GitHub.io</a>
 </p>        
<p style="text-align: left">
© Olivier Cuisenaire, 2018 </p>
</td>
</tr>
</table>